In [8]:
import six
import numpy as np
import datetime
import os
import glob
import matplotlib.pyplot as plt
import logging as log
from stompy import utils
from stompy import memoize
import xarray as xr
from matplotlib import colors
%matplotlib notebook

In [9]:
from stompy.grid import unstructured_grid
six.moves.reload_module(unstructured_grid)
import postprocess_v00 as post
six.moves.reload_module(post)

<module 'postprocess_v00' from '/home/rusty/src/microplastic_sfbay/postprocess/postprocess_v00.py'>

In [3]:
grid_fn="/opt2/sfb_ocean/suntans/runs/merged_018_20171227/ptm_average.nc_0000.nc"  
grid=post.grid_from_ptm_hydro(grid_fn)

Process one month chunks
==

In [23]:
out_dir="processed"
os.path.exists(out_dir) or os.makedirs(out_dir)

def sw_conc_func(group,src,behavior):
    
def process_batch(ptm_runs,
                  time_range,
                  patterns,
                  z_ranges,
                  max_age_days=15,
                  version='v04'):
    # v03: shift to particles/m3 units (in sync with change in postprocess_v00).
    #   to speed things up, simply adjust v02 output if that exists.
    # v04: should be same as v03, but is recomputed, not just post-hoc scaled.
    time_str=(utils.to_datetime(time_range[0]).strftime('%Y%m%d')
              + '_'
              + utils.to_datetime(time_range[1]).strftime('%Y%m%d'))
    
    for group_name,group_patt in patterns:
        log.info(f"Processing {group_name}, pattern: {group_patt}")
        chunk_dir=os.path.join(out_dir,time_str,group_name)
        os.path.exists(chunk_dir) or os.makedirs(chunk_dir)
  
        # calculated on-demand in the loop below
        @memoize.memoize()
        def parts():
            log.info("Extracting particles")
            result=post.query_runs(ptm_runs,
                                   group_patt=group_patt,
                                      time_range=time_range,
                                      z_range=None, # not ready
                                      max_age=np.timedelta64(max_age_days,'D'),
                                      conc_func=post.conc_func,
                                      grid=grid)
            log.info("Adding vertical info")
            result=post.add_z_info(result,grid,ptm_runs)
            return result
        for z_name,z_range in z_ranges: 
            # Just the particles for the period, with mass, but not filtered
            # on elevation:
            conc_nc_fn=os.path.join(chunk_dir,f'particles-{z_name}-{max_age_days}days-{version}.nc')
            #old_conc_nc_fn=os.path.join(chunk_dir,f'particles-{z_name}-{max_age_days}days-v02.nc')
            #if not os.path.exists(conc_nc_fn) and os.path.exists(old_conc_nc_fn):
            #    ds=xr.open_dataset(old_conc_nc_fn)
            #    ds.conc.values *= 1000 # scale up to account for old code that used particles/L
            #    ds.conc.attrs['units']='particles m-2'
            #    ds.to_netcdf(conc_nc_fn)
            #    ds.close()
            #    log.info("Rescaled v02 output to v03")
            if not os.path.exists(conc_nc_fn):
                log.info(f"writing to {conc_nc_fn}")
                p=parts()
                p=post.filter_by_z_range(p,z_range,grid,ptm_runs)
                conc=post.particle_to_density(p,grid,normalize='area')
                # could also use the z_bed, z_surf values to turn particle mass into
                # a mass/z, such that normalize by area then gives a volume concentration.
                # unless it's full water column, have to do some truncating
                
                # this should preserve most of the metadata
                ds=p.copy()
                particle_vars=[v for v in ds.variables if 'particle' in ds[v].dims]
                for v in particle_vars:
                    del ds[v]
                
                grid.write_to_xarray(ds=ds)
                ds['conc']=('face',),conc
                ds['conc'].attrs['units']='particles m-2'
                
                ds.to_netcdf(conc_nc_fn)
                log.info("done writing")

In [24]:
patterns=[
    ('-0.05','.*_down50000'),
    ('-0.005','.*_down5000'),
    ('-0.0005','.*_down500'),
    ('0.0','.*_none'),
    ('0.0005','.*_up500'),
    ('0.005','.*_up5000'),
    ('0.05','.*_up50000')
]

z_ranges=[
    ('bed',[0,0.5]),
    ('surf',[-0.5,0]),
    ('avg',[0,0])
]

In [25]:
def process_date(run_date):
    """
    run_date: YYYYMMDD string for start of runs
    """
    ptm_runs=[post.PtmRun(run_dir=d) 
              for d in glob.glob(f"/opt2/sfb_ocean/ptm/all_source/{run_date}/*") ]
    assert len(ptm_runs)==7
    
    # just the time period with a full field for max_age=15D
    start=np.timedelta64(15,'D') + utils.to_dt64(datetime.datetime.strptime(run_date,'%Y%m%d'))

    time_range=[start,start+np.timedelta64(15,'D')]
    process_batch(ptm_runs,time_range,patterns,z_ranges=z_ranges)    

In [26]:
for d in ["20170615",
          "20170715",
          "20170815",
          "20170915",
          "20171015",
          "20171115",
          "20171215",
          "20180115",
          "20180215",
          "20180315",
          "20180415"
         ]:
    log.info("-"*20 + d + "-"*20)
    process_date(d)

INFO:root:--------------------20170615--------------------
INFO:root:Processing -0.05, pattern: .*_down50000
INFO:root:writing to processed/20170630_20170715/-0.05/particles-bed-15days-v04.nc
INFO:root:Extracting particles
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170615/w-0.05: sunnyvale_down50000
INFO:root:Will skip source sunnyvale, behavior down50000, its concentration is 0
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170615/w-0.05: src001_down50000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170615/w-0.05: petaluma_down50000
INFO:root:Will skip source petaluma -- it's in skip_source
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170615/w-0.05: src000_down50000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170615/w-0.05: SCLARAVCc_down50000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170615/w-0.05: UALAMEDA_down50000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170615/w-0.05: NAPA_down50000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170615/w-0.05: san_jose_down50000
INFO:root:/opt2/sfb_ocean/p

INFO:root:Will skip source src001, behavior none, its concentration is 0
INFO:root:Adding vertical info
INFO:UnstructuredGrid:Building point index (2743495 points)
INFO:UnstructuredGrid:Querying point index (57124 cells)
INFO:root:done writing
INFO:root:writing to processed/20170630_20170715/0.0/particles-surf-15days-v04.nc
INFO:root:done writing
INFO:root:writing to processed/20170630_20170715/0.0/particles-avg-15days-v04.nc
INFO:utils:2065614
INFO:root:done writing
INFO:root:Processing 0.0005, pattern: .*_up500
INFO:root:writing to processed/20170630_20170715/0.0005/particles-bed-15days-v04.nc
INFO:root:Extracting particles
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170615/w0.0005: sunnyvale_up500
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170615/w0.0005: san_jose_up500
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170615/w0.0005: fs_up500
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170615/w0.0005: cccsd_up500
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170615/w0.0005: palo_alto_up500
INF

INFO:root:Adding vertical info
INFO:UnstructuredGrid:Building point index (6035689 points)
INFO:UnstructuredGrid:Querying point index (57124 cells)
INFO:utils:2151945
INFO:root:done writing
INFO:root:writing to processed/20170730_20170814/-0.05/particles-surf-15days-v04.nc
INFO:root:done writing
INFO:root:writing to processed/20170730_20170814/-0.05/particles-avg-15days-v04.nc
INFO:utils:2143733
INFO:utils:4278209
INFO:root:done writing
INFO:root:Processing -0.005, pattern: .*_down5000
INFO:root:writing to processed/20170730_20170814/-0.005/particles-bed-15days-v04.nc
INFO:root:Extracting particles
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170715/w-0.005: UALAMEDA_down5000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170715/w-0.005: petaluma_down5000
INFO:root:Will skip source petaluma -- it's in skip_source
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170715/w-0.005: NAPA_down5000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170715/w-0.005: SCLARAVCc_down5000
INFO:root:/opt2/sfb_ocean/ptm/

INFO:UnstructuredGrid:Building point index (6584195 points)
INFO:UnstructuredGrid:Querying point index (57124 cells)
INFO:root:done writing
INFO:root:writing to processed/20170730_20170814/0.0005/particles-surf-15days-v04.nc
INFO:utils:2087193
INFO:root:done writing
INFO:root:writing to processed/20170730_20170814/0.0005/particles-avg-15days-v04.nc
INFO:utils:2095423
INFO:utils:4196119
INFO:utils:6282421
INFO:root:done writing
INFO:root:Processing 0.005, pattern: .*_up5000
INFO:root:writing to processed/20170730_20170814/0.005/particles-bed-15days-v04.nc
INFO:root:Extracting particles
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170715/w0.005: SacRiver_up5000
INFO:root:Will skip source SacRiver, behavior up5000, its concentration is 0
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170715/w0.005: palo_alto_up5000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170715/w0.005: sunnyvale_up5000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170715/w0.005: SCLARAVCc_up5000
INFO:root:/opt2/sfb_ocean/ptm/al

INFO:utils:4039029
INFO:root:done writing
INFO:root:writing to processed/20170830_20170914/-0.005/particles-surf-15days-v04.nc
INFO:root:done writing
INFO:root:writing to processed/20170830_20170914/-0.005/particles-avg-15days-v04.nc
INFO:utils:2053056
INFO:utils:4128921
INFO:utils:6260889
INFO:root:done writing
INFO:root:Processing -0.0005, pattern: .*_down500
INFO:root:writing to processed/20170830_20170914/-0.0005/particles-bed-15days-v04.nc
INFO:root:Extracting particles
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170815/w-0.0005: src001_down500
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170815/w-0.0005: palo_alto_down500
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170815/w-0.0005: src000_down500
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170815/w-0.0005: san_jose_down500
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170815/w-0.0005: fs_down500
INFO:root:/opt2/sfb_ocean/ptm/all_source/20170815/w-0.0005: SJRiver_down500
INFO:root:Will skip source SJRiver, behavior down500, its concentrat

...

INFO:utils:2047475
INFO:utils:4139868
INFO:root:done writing
INFO:root:--------------------20171015--------------------
INFO:root:Processing -0.05, pattern: .*_down50000
INFO:root:writing to processed/20171030_20171114/-0.05/particles-bed-15days-v04.nc
INFO:root:Extracting particles
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171015/w-0.05: sunnyvale_down50000
INFO:root:Will skip source sunnyvale, behavior down50000, its concentration is 0
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171015/w-0.05: src001_down50000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171015/w-0.05: petaluma_down50000
INFO:root:Will skip source petaluma -- it's in skip_source
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171015/w-0.05: src000_down50000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171015/w-0.05: SCLARAVCc_down50000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171015/w-0.05: UALAMEDA_down50000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171015/w-0.05: NAPA_down50000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20

INFO:root:writing to processed/20171030_20171114/0.0/particles-surf-15days-v04.nc
INFO:root:done writing
INFO:root:writing to processed/20171030_20171114/0.0/particles-avg-15days-v04.nc
INFO:utils:2016857
INFO:root:done writing
INFO:root:Processing 0.0005, pattern: .*_up500
INFO:root:writing to processed/20171030_20171114/0.0005/particles-bed-15days-v04.nc
INFO:root:Extracting particles
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171015/w0.0005: sunnyvale_up500
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171015/w0.0005: san_jose_up500
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171015/w0.0005: fs_up500
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171015/w0.0005: cccsd_up500
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171015/w0.0005: palo_alto_up500
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171015/w0.0005: src001_up500
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171015/w0.0005: UALAMEDA_up500
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171015/w0.0005: SJRiver_up500
INFO:root:Will skip source S

INFO:root:writing to processed/20171130_20171215/-0.005/particles-bed-15days-v04.nc
INFO:root:Extracting particles
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171115/w-0.005: UALAMEDA_down5000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171115/w-0.005: petaluma_down5000
INFO:root:Will skip source petaluma -- it's in skip_source
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171115/w-0.005: NAPA_down5000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171115/w-0.005: SCLARAVCc_down5000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171115/w-0.005: src002_down5000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171115/w-0.005: fs_down5000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171115/w-0.005: SJRiver_down5000
INFO:root:Will skip source SJRiver, behavior down5000, its concentration is 0
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171115/w-0.005: cccsd_down5000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171115/w-0.005: SacRiver_down5000
INFO:root:Will skip source SacRiver, behavior down5000, its concentrat

INFO:root:Extracting particles
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171115/w0.005: SacRiver_up5000
INFO:root:Will skip source SacRiver, behavior up5000, its concentration is 0
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171115/w0.005: palo_alto_up5000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171115/w0.005: sunnyvale_up5000
INFO:root:/opt2/sfb_ocean/ptm/all_source/20171115/w0.005: SCLARAVCc_up5000


KeyboardInterrupt: 

In [ ]:
class BayConcFigure(object):
    figsize=(9,7)
    vmin=1e-6
    vmax=1e-2
    zoom=(517521., 613202., 4139744., 4230105.)
    cax_loc=[0.7,0.25,0.03,0.35]
    num=None
    def __init__(self,conc,**kw):
        utils.set_keywords(self,kw)
                    
        self.fig=plt.figure(figsize=(10,8),num=self.num)
        self.ax=self.fig.add_subplot(1,1,1)
        
        self.ccoll=grid.plot_cells(values=conc.clip(self.vmin,self.vmax),
                                   cmap='jet',norm=colors.LogNorm(vmin=self.vmin,vmax=self.vmax),
                                   edgecolor='face',lw=0.4,ax=self.ax)
        self.cax=self.fig.add_axes(self.cax_loc)
        plt.colorbar(ccoll,cax=self.cax)
        self.ax.set_aspect('equal')
        self.ax.xaxis.set_visible(0)
        self.ax.yaxis.set_visible(0)
        self.ax.axis(self.zoom)
        self.fig.subplots_adjust(left=0.01,right=0.99,top=0.99,bottom=0.01)
        